In [13]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim, logging
import string
import random
# Sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
import multiprocessing
import time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.cross_validation import train_test_split

/Users/akashbachu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
newsgroup = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(newsgroup.target_names.unique())

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


In [8]:
def unwantedstuff(text):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(edu|re|subject|from|a|an|the)\b', ' ', text)
    
    def rem_numbers(text):
        return re.sub(r'\b[0-9]\b',' ', text)
    
    def stripnumbers(text):
        return re.sub(r'\d+', '', text)
    
#     def strip_digits(text):
#         return re.sub("\d+", "", text)
        

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    def remstopwords(text):
        stop_words = stopwords.words('english')
        #stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
        [word for word in text if word not in stop_words]
        return text
    

    return remstopwords(rem_numbers(stripnumbers(white_space_fix(remove_articles(remove_punc(lower(text)))))))

In [9]:
newsgroup.content = newsgroup.content.apply(unwantedstuff)

In [20]:
newsgroup.head()

,content,target,target_names
0,lerxstwamumdedu wheres my thing what car is th...,7,rec.autos
1,guykuocarsonuwashingtonedu guy kuo si clock po...,4,comp.sys.mac.hardware
10,irwincmptrclonestarorg irwin arnstein recommen...,8,rec.motorcycles
100,tchenmagnusacsohiostateedu tsungkun chen softw...,6,misc.forsale
1000,dablnlmnihgov don ab lindbergh diamond ssx win...,2,comp.os.ms-windows.misc


In [14]:
X_train, X_test, y_train, y_test = train_test_split(newsgroup.content, newsgroup.target_names, test_size=0.3)

In [16]:
X_train.head()

8032    hahietanentnclustelenokiafi tps will stay on t...
5829    dbdurartusdpaorg david davidian accounts of an...
5558    rgaschnloraclecom robert gasch overriding defa...
6520    daonscclehighedu donald oshall original ibm pc...
2507    rayengrlatechedu bill ray bible and abortion o...
Name: content, dtype: object

In [17]:
X_test.head()

4629     etxonssufsaericssonse staffan axelsson nhl swe...
1890     organization city university of new york fsicu...
5658     alescomcom al donaldson once tapped your code ...
2311     dlplilutacuk dlp li new svga card replyto dlpl...
10227    turmoilhalcyoncom tim crowley waco clinton pre...
Name: content, dtype: object

In [18]:
y_train.head()

8032            rec.sport.hockey
5829       talk.politics.mideast
5558              comp.windows.x
6520    comp.sys.ibm.pc.hardware
2507                 alt.atheism
Name: target_names, dtype: object

In [19]:
y_test.head()

4629             rec.sport.hockey
1890           talk.politics.misc
5658                    sci.crypt
2311     comp.sys.ibm.pc.hardware
10227          talk.politics.guns
Name: target_names, dtype: object

In [21]:
# Create tagged documents
news_tagged_documents = []

for i, document in enumerate(newsgroup.content):
    news_tagged_documents.append(TaggedDocument(document.split(' '), [i]))
    
# Train Doc2Vec
doc2vec_vector_size = 100

doc2vec_model = Doc2Vec(documents=news_tagged_documents, size=doc2vec_vector_size, window=3, 
                        min_count=2, iter=30, workers=multiprocessing.cpu_count())
doc2vec_model.init_sims(replace=False)

# Vectorize documents and split the whole corpus in train and test sets
news_vectors = []

for document in newsgroup.content:
    news_vectors.append(doc2vec_model.infer_vector(document.split(' ')))

d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(news_vectors, newsgroup.target_names, test_size=0.30)

/Users/akashbachu/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/akashbachu/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-05-03 21:47:15,322 INFO collecting all words and their counts
2018-05-03 21:47:15,324 INFO PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-03 21:47:15,932 INFO PROGRESS: at example #10000, processed 2528635 words (4178370/s), 111896 word types, 10000 tags
2018-05-03 21:47:16,026 INFO collected 118063 word types and 11314 unique tags from a corpus of 11314 examples and 2875130 wor

In [28]:
news_tagged_documents[0:10]

[TaggedDocument(words=['lerxstwamumdedu', 'wheres', 'my', 'thing', 'what', 'car', 'is', 'this', 'nntppostinghost', 'racwamumdedu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', '', 'i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'late', 's', 'early', 's', 'it', 'was', 'called', 'bricklin', 'doors', 'were', 'really', 'small', 'in', 'addition', 'front', 'bumper', 'was', 'separate', 'rest', 'of', 'body', 'this', 'is', 'all', 'i', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'email', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst'], tags=[0]),
 TaggedDocument(words=['guykuocarsonuwashingt

In [29]:
news_vectors[0:10]

[array([ 0.4012155 ,  0.29046327, -0.26900208, -0.42005745, -0.03631262,
        -0.23612992, -0.5538636 , -0.55789834, -0.8562401 , -0.39409778,
        -0.56899273, -0.18662849, -0.26644275,  0.00900707, -0.24209653,
        -0.02695803,  0.7740409 ,  0.42278603, -0.235645  , -0.18970582,
        -0.49766052,  0.5573119 ,  0.01429904, -0.08607168, -0.5332963 ,
        -0.02337614, -0.49744815, -0.05459675,  0.04612977, -0.35946766,
         0.64605606,  0.19039525, -0.27039596, -0.10652188, -0.66636914,
        -0.34752372, -0.46515653,  0.10278156,  0.39243373,  0.55434394,
        -0.09884247,  0.07233556, -0.371096  ,  0.015388  , -0.12525415,
         0.31090677, -0.49080294,  0.08297917, -0.08027903, -0.02933257,
         0.15362751,  0.8833    ,  0.3332273 ,  0.02835109,  0.4675931 ,
         0.16012892, -0.64082795, -0.5720117 ,  0.24229325,  0.41147313,
         0.28047052,  0.6730288 ,  0.39164987, -0.03754878,  0.21191438,
        -0.6194759 ,  0.18077901, -0.16750148, -0.1

In [32]:
d2v_X_train[0:10]

[array([ 0.2418207 ,  0.10940918,  2.8991206 ,  0.00679084, -0.37315407,
        -0.5255526 , -1.0028869 ,  0.23362379, -0.09806396,  1.599048  ,
         0.95804983, -1.4578683 , -1.0609597 ,  1.2320396 , -0.29483742,
         0.06089968, -0.7739798 ,  0.40279135,  1.5525174 ,  1.921918  ,
         0.610247  ,  1.710076  ,  1.7222391 , -0.56656563, -0.9516793 ,
        -1.9434495 , -2.3133318 , -0.7219837 ,  0.7144775 , -1.8487737 ,
        -0.07389559, -1.1107057 , -0.9194918 , -1.766121  , -0.03950711,
        -1.0611577 , -0.5010649 ,  1.7431126 ,  1.3839912 ,  1.6195779 ,
         0.43201244, -1.1765336 , -0.6815334 ,  0.00694499, -1.5161439 ,
         1.8671254 ,  0.977931  ,  1.9414984 , -0.7422945 , -2.3130877 ,
        -0.12224036,  0.8572679 , -0.57459825,  0.91811603,  1.1431758 ,
         0.11162429, -1.291704  , -1.3232656 ,  0.79114395, -0.05368081,
         0.4687098 , -0.22034448, -1.1565605 ,  1.0811026 ,  0.6572089 ,
        -0.48562896,  0.8891662 , -0.57832706,  0.5

In [33]:
d2v_X_test[0:10]

[array([-0.13862312,  0.6487795 ,  0.28222227, -0.92608154, -0.04101097,
        -0.37924635, -0.28182387, -0.42371964,  1.0631053 , -0.99948233,
        -0.5409142 ,  0.25230145, -0.3168472 ,  0.5369217 , -0.68773997,
        -0.37051815,  0.74849206,  0.12880763, -0.71417385, -0.16597177,
        -0.73659813,  0.5534072 , -0.72975415,  0.24093653, -0.1567864 ,
        -0.4037402 , -0.7553576 ,  1.2437068 ,  0.45042676,  0.19834916,
         0.31777155,  0.02997895, -0.6823927 ,  0.79883224,  0.5222684 ,
        -0.40872598,  0.4232849 ,  0.3679576 ,  0.01856967,  0.5408001 ,
         0.29970896,  0.49521038, -0.8180413 , -0.1259284 , -0.2171726 ,
         0.36192265, -0.6974732 , -0.19468293,  0.31028384,  0.12175668,
        -0.35716817, -0.204136  ,  0.27496642, -0.10523734,  0.75842524,
        -0.26995036, -0.06995723,  0.19819655,  0.2947884 , -0.07704976,
         0.70722896,  0.68534493, -0.02260355,  0.50161237, -0.5099274 ,
        -1.1550181 ,  0.3506459 ,  0.3403843 , -1.1

In [34]:
d2v_Y_train[0:10]

5913       talk.politics.mideast
7388             sci.electronics
8481             rec.motorcycles
6139    comp.sys.ibm.pc.hardware
7023             rec.motorcycles
2259                misc.forsale
8091      soc.religion.christian
9121     comp.os.ms-windows.misc
2214                     sci.med
9141            rec.sport.hockey
Name: target_names, dtype: object

In [35]:
d2v_Y_test[0:10]

5199    comp.sys.ibm.pc.hardware
6712       comp.sys.mac.hardware
7623      soc.religion.christian
4457                     sci.med
6608          talk.religion.misc
5482    comp.sys.ibm.pc.hardware
1002          talk.politics.misc
5347     comp.os.ms-windows.misc
9917       talk.politics.mideast
5155       talk.politics.mideast
Name: target_names, dtype: object

In [36]:
scaler = StandardScaler()
svc = SVC()

# Define pipeline
svc_d2v_pipeline = Pipeline([('scaler', scaler),
                             ('svc', svc)])

# Grid search parameters
svc_d2v_param_grid = [
    { 
        'svc__kernel': [ 'linear' ],
        'svc__C': [0.5, 1.0, 5.0]
    },
]

# Perform grid search
svc_d2v_gs = GridSearchCV(estimator=svc_d2v_pipeline, param_grid=svc_d2v_param_grid, 
                           scoring='accuracy', cv=25, n_jobs=multiprocessing.cpu_count())

svc_d2v_gs.fit(d2v_X_train, d2v_Y_train)

GridSearchCV(cv=25, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid=[{'svc__kernel': ['linear'], 'svc__C': [0.5, 1.0, 5.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [37]:
y_pred = svc_d2v_gs.predict(d2v_X_test)
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))   

0.047799529430996676
0.04760062527671062
0.04861602237864948


In [38]:
svc_d2v_best = svc_d2v_gs.best_estimator_
svc_d2v_best.fit(d2v_X_train, d2v_Y_train)
print('Best model: %s' % str(svc_d2v_gs.best_params_))
print('Best score: %f' % svc_d2v_gs.best_score_)
print('Best test accuracy: %f' % svc_d2v_best.score(X_test, y_test))

Best model: {'svc__C': 0.5, 'svc__kernel': 'linear'}
Best score: 0.612325


ValueError: could not convert string to float: 'steinlytopazucscedu steinn sigurdsson commercial mining activities on moon organization lick observatoryuco lines  aprcsrochesteredu steinlyaprtopazucscedu aprcsrochesteredusteinlyaprtopazucscedu rjinnjmojoengumdedu nntppostinghost topazucscedu inreplyto sysmgrkingengumdedus message of  apr   gmt in article rjinnjmojoengumdedu sysmgrkingengumdedu doug mohney writes in article steinlyaprtopazucscedu steinlytopazucscedu steinn sigurdsson writes very cost effective if you use right accounting method sherzer methodology hell yes im not going to let bunch of seven suits tell me what right way to estimate cost effectiveness is at least not until they can make their mind up long enough to leave their scheme stable for fiscal year or two seriously though if you were to ask british government whether their colonisation efforts in americas were cost effective what answer do you think youd get what if you asked in     and  respectively steinn sigurd